### 本代码块将介绍物体检测入门

物体检测的关键工具就是颜色空间变化  
即**cv2.cvtColor()函数**<br>
常用的变化一般只有两个:
<ol>
<strong>
<li>cv2.COLOR_BGR2BGRA</li>
<li>cv2.COLOR_BGR2HSV</li>
</ol>

### 物体跟踪入门基本步骤  
现在我们知道怎样将一幅图像从 BGR 转换到 HSV 了，我们可以利用这  
一点来提取带有某个特定颜色的物体。在 HSV 颜色空间中要比在 BGR 空间  
中更容易表示一个特定颜色。在我们的程序中，我们要提取的是一个蓝色的物  
体。下面就是就是我们要做的几步： <strong> 
1. • 从视频中获取每一帧图像
2. • 将图像转换到 HSV 空间
3. • 设置 HSV 阈值到蓝色范围。
4. • 获取蓝色物体，当然我们还可以做其他任何我们想做的事，比如：在蓝色  
      物体周围画一个圈。


In [2]:
#以红笔为例 两个主要rgb值(152,70,82),(141,71,87)

import cv2
import numpy as np
import matplotlib.pyplot as plt

#展示函数
def show(img,title = 'Track'):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#根据给定HSV图像确定上下界HSV
def cvt_hsv(hsv):
    b = hsv[0][0][0]
    l_hsv = np.uint8([[[b-10,100,100]]])
    h_hsv = np.uint8([[[b+10,255,255]]])
    ans = [l_hsv,h_hsv]
    return ans

pen = cv2.imread('images/red_pen.jpg')

#先要确定被追踪对象的HSV值,直接用cvt函数
#因为彩色转换只能对(w,h,3)图像生效
#错误写法
track_color = np.uint8([82,70,152])#注意opencv是默认是BGR
print(track_color.shape)

#正确写法
track_color = np.uint8([[[82,70,152]]])
print(track_color.shape)
track_hsv = cv2.cvtColor(track_color,cv2.COLOR_BGR2HSV)
print(track_hsv)

#以hsv设定hsv颜色空间对应物体的阈值
l_hsv,h_hsv = cvt_hsv(track_hsv)
print(l_hsv)
print(h_hsv)

#开始进行物体追踪
cap = cv2.VideoCapture(0)
if cap.isOpened() == False:
    print('Error')

while(True):
    ret,frame = cap.read()
    if ret == False:
        break
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    #难点1:暂时无法理解原理
    #根据阈值建立掩膜(可以理解为返回接收了一个阈值图像,即满足特定阈值范围的才显示)
    mask = cv2.inRange(hsv,l_hsv,h_hsv)

    #难点2:对原图像与掩膜进行位运算
    res = cv2.bitwise_and(frame,frame,mask = mask)

    #out = np.hstack([frame,mask,res])
    cv2.imshow('Origin',frame)
    cv2.imshow('Mask',mask)
    cv2.imshow('Track',res)
    if cv2.waitKey(10)&0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


(3,)
(1, 1, 3)
[[[176 138 152]]]
[[[166 100 100]]]
[[[186 255 255]]]


## HSV颜色模型的理解
HSV颜色空间

HSV颜色空间由三个分量组成：**色相（Hue）、饱和度（Saturation）和明度（Value）**。通过调整这些分量的值，我们可以选择特定的颜色范围，HSV具体含义如下：

**色相（Hue）**：色相值表示颜色在颜色轮上的位置。不同的颜色在色相上有不同的取值范围。例如，红色的色相值约为0-10或160-180，蓝色的色相值约为110-130，绿色的色相值约为50-70。根据所需识别的颜色，选择相应的色相范围。

**饱和度（Saturation）**：饱和度值表示颜色的纯度或鲜艳程度。较高的饱和度值表示颜色更加鲜艳，而较低的饱和度值表示颜色较为灰暗。根据实际场景中颜色的饱和度，选择适当的饱和度范围。

**明度（Value）**：明度值表示颜色的亮度或明暗程度。较高的明度值表示颜色较亮，而较低的明度值表示颜色较暗。根据实际场景中颜色的明度，选择适当的明度范围。

这些值是根据经验和实验进行调整的，以获得最佳的颜色识别效果。我们可以根据具体的应用场景和需求进行调整，以适应不同的颜色识别任务。在实际使用中，我们可能需要进行多次尝试和调整，以找到最适合需求的颜色范围。

对于每种颜色（红色、蓝色和绿色），我们使用np.array()函数创建了一个包含三个元素的NumPy数组。这些元素分别代表HSV颜色空间中的色相、饱和度和明度的下限和上限。
————————————————

                版权声明：本文为博主原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接和本声明。  
                原文链接:https://blog.csdn.net/m0_73629902/article/details/136034641

### 掩膜的理解
在进行颜色识别时，我们通过创建掩膜的方法来将图像进行**二值化**，即非黑即白，通过创建颜色掩膜，  
可以将图像中的特定颜色范围提取出来，让摄像头对目标颜色更为敏感。

具体做法为:**我们使用cv2.inRange()函数来创建颜色掩膜。它会根据指定的颜色范围，将图像中在范围内的像素设置为255（白色），而其他像素则设置为0（黑色）**

In [15]:
#根据上面代码进行增强,即上面只用了一个红笔颜色,现在加上两种主要的
#两个主要rgb值深:(152,70,82),浅:(141,71,87),(123,65,77),灰色:(158,158,156)

import cv2
import matplotlib.pyplot as plt
import numpy as np

def cvt_hsv(hsv):
    b = hsv[0][0][0]
    l_hsv = np.uint8([[[b-10,100,100]]])
    h_hsv = np.uint8([[[b+10,255,255]]])
    ans = [l_hsv,h_hsv]
    return ans

bgr1 = np.array([[[82,70,152]]],np.uint8)
bgr2 = np.array([[[156,158,158]]],np.uint8)

hsv1 = cv2.cvtColor(bgr1,cv2.COLOR_BGR2HSV)
hsv2 = cv2.cvtColor(bgr2,cv2.COLOR_BGR2HSV)

l_hsv1,h_hsv1 = cvt_hsv(hsv1)
l_hsv2,h_hsv2 = cvt_hsv(hsv2)
print(l_hsv1,h_hsv1)
print(l_hsv2,h_hsv2)

cap = cv2.VideoCapture(0)

if cap.isOpened() == False:
    print('Error!')

while(True):
    ret,frame = cap.read()
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv,l_hsv1,h_hsv1)
    mask2 = cv2.inRange(hsv,l_hsv2,h_hsv2)

    #进行cv2的图像加法(饱和操作)
    mask = cv2.add(mask1,mask2)

    res = cv2.bitwise_and(frame,frame,mask = mask)
    cv2.imshow('Res',res)
    if cv2.waitKey(10)&0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

[[[166 100 100]]] [[[186 255 255]]]
[[[ 20 100 100]]] [[[ 40 255 255]]]
